# Using `kugupu` to calculate molecular coupling networks WITH ML

This notebook demonstrates how to calculate molecular coupling between fragments, inspect the results and save and load these results to file.  These results files will be the basis of all further analysis done using the `kugupu` package.

This will require version 0.20.0 of MDAnalysis, and kugupu to be installed.

In [1]:
import MDAnalysis as mda
import numpy as np
import kugupu as kgp
from openff.toolkit.topology import Atom
import xtb

models available
{'ocelotml': <class 'kugupu.ocelotl_model.OcelotMLModel'>, 'yaehmop': <class 'kugupu.yaehmop.YaehmopModel'>}


Firstly we create an `MDAnalysis.Universe` object from our simulation files:

In [2]:
u = mda.Universe('datafiles/C6.data', 'datafiles/C6.dcd')

/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


This system has 46,500 atoms in 250 different fragments.

In [3]:
print(u.atoms.n_atoms, len(u.atoms.fragments))

46500 250


Our dynamics simulation has 5 frames of results.

In [4]:
print(u.trajectory.n_frames)

5


In [5]:
# u.atoms.fragments[0].names

To perform the coupling calculations our `Universe` will require bond information (for determining fragments) and element information (for the tight binding calculations) stored inside the `.names` attribute.

Our Lammps Data file did not include element symbols, so we can add these to the Universe now...

In [6]:
def add_names(u):
    # Guesses atom names based upon masses
    def approx_equal(x, y):
        return abs(x - y) < 0.1
    
    # mapping of atom mass to element
    massdict = {}
    for m in set(u.atoms.masses):
        for elem, elem_mass in mda.guesser.tables.masses.items():
            if approx_equal(m, elem_mass):
                massdict[m] = elem
                break
        else:
            raise ValueError
            
    u.add_TopologyAttr('names')
    for m, e in massdict.items():
        u.atoms[u.atoms.masses == m].names = e
        

add_names(u)

In [9]:
from openff.toolkit.topology import Molecule
from MDAnalysis.topology.guessers import guess_types
from kugupu._yaehmop import shift_dimer_images
from MDAnalysis.lib.mdamath import triclinic_vectors
from kugupu.dimers import find_dimers
from MDAnalysis.transformations.wrap import unwrap

TEST = 200

elements = guess_types(u.atoms.names)
u.add_TopologyAttr("elements", elements)

ag = u.atoms
transform = mda.transformations.unwrap(ag)
u.trajectory.add_transformations(transform)

# print(u.dimensions.shape)
example_fragments = u.atoms.fragments
dimers_list = find_dimers(example_fragments, cutoff=5.0)
list_dimers = sorted(dimers_list.items())
frag_i, frag_j =list_dimers[TEST][-1]
shifted_coords = shift_dimer_images(frag_i, frag_j)
total_frag = frag_i + frag_j
total_frag.positions = shifted_coords
rdkit = total_frag.convert_to.rdkit()
mol = Molecule.from_rdkit(rdkit, allow_undefined_stereo=True)
mol


/var/folders/8_/xls29m695yl7qglq81r94h7w0000gr/T/ipykernel_36723/2561346940.py:10: DeprecationWarning: `guess_types` is deprecated!
`guess_types` will be removed in release 3.0.0.
MDAnalysis.topology.guessers is deprecated in favour of the new Guessers API. See MDAnalysis.guesser.default_guesser for more details.
  elements = guess_types(u.atoms.names)
/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.10/site-packages/MDAnalysis/topology/guessers.py:184: DeprecationWarning: `guess_atom_element` is deprecated!
`guess_atom_element` will be removed in release 3.0.0.
MDAnalysis.topology.guessers is deprecated in favour of the new Guessers API. See MDAnalysis.guesser.default_guesser for more details.
  [guess_atom_element(name) for name in atom_names], dtype=object
2025-06-12T16:12:10.136322+0100 INFO Finding dimers within 5.0, passed 250 fragments
2025-06-12T16:12:10.481355+0100 INFO Found 3282 dimers


shift


/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.10/site-packages/openff/toolkit/topology/molecule.py:4411: MultipleComponentsInMoleculeWarning: RDKit Molecule passed to from_rdkit consists of more than one molecule, consider running rdkit.Chem.AllChem.GetMolFrags(rdmol, asMols=True) or splitting input SMILES at '.' to get separate molecules and pass them to from_rdkit one at a time. While this is supported for legacy reasons, OpenFF Molecule objects are not supposed to contain disconnected chemical graphs and this may result in undefined behavior later on. The OpenFF ecosystem is built to handle multiple molecules, but they should be in a Topology object, ex: top = Topology.from_molecules([mol1, mol2])
  molecule = toolkit.from_rdkit(


NGLWidget()

In [8]:
from qcportal.singlepoint import SinglepointDriver
from openff.qcsubmit.factories import BasicDatasetFactory
from openff.qcsubmit.common_structures import QCSpec
from qcelemental.models.results import WavefunctionProtocolEnum
from qcportal import PortalClient


USERNAME = "charlie"
PASSWORD = "kuano123"
ADDRESS ="http://localhost:7777"

client = PortalClient(
    address=ADDRESS, username = USERNAME, password= PASSWORD
)



Connection error for http://localhost:7777/auth/v1/login: HTTPConnectionPool(host='localhost', port=7777): Max retries exceeded with url: /auth/v1/login (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x322776b30>: Failed to establish a new connection: [Errno 61] Connection refused')) - retrying in 0.50 seconds [1/5]
Connection error for http://localhost:7777/auth/v1/login: HTTPConnectionPool(host='localhost', port=7777): Max retries exceeded with url: /auth/v1/login (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x322776d10>: Failed to establish a new connection: [Errno 61] Connection refused')) - retrying in 0.98 seconds [2/5]
Connection error for http://localhost:7777/auth/v1/login: HTTPConnectionPool(host='localhost', port=7777): Max retries exceeded with url: /auth/v1/login (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x3227766e0>: Failed to establish a new connection: [Errno 61] Connection ref

ConnectionRefusedError: 

Could not connect to server http://localhost:7777/, please check the address and try again.

In [ ]:
factory = BasicDatasetFactory(
    driver=SinglepointDriver.energy,
    qc_specifications={
        "xtb": QCSpec(
            program="xtb",
            method="gfn0xtb",
            basis=None,
            spec_name="ani1ccx",
            spec_description="ANI1ccx standard specification",
            store_wavefunction=WavefunctionProtocolEnum.all
        ),
    },
)
molecules = [mol]
factory.compute_tag = 'xtb'
dataset = factory.create_dataset(
    dataset_name="xtb_test",
    description="xtb test for coupling",
    tagline="test dataset for xtb wavefunction",
    molecules=molecules,  #I deleted
)
dataset.submit(
    client = client,
)

Preparation                   :   0%|                     | 0/1 [00:00<?, ?it/s]/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.10/site-packages/openff/qcsubmit/utils/smirnoff.py:196: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_conformer[index_mapping[i]] = conformer[i]
Preparation                   : 100%|█████████████| 1/1 [00:00<00:00,  2.31it/s]
'metadata' parameter has been deprecated and will be removed in a future version. Use 'extras' instead
'tag' is deprecated; use 'compute_tag' instead
'priority' is deprecated; use 'compute_priority' instead


InsertCountsMetadata(n_inserted=1, n_existing=0, error_description=None, errors=[])

In [ ]:
numbers = np.array([atom.atomic_number for atom in mol.atoms])
positions = mol.conformers[0]
positions.m

array([[103.76624298, 140.33729553,  68.82037354],
       [104.44104767, 139.38671875,  69.59480286],
       [105.59061432, 138.71580505,  69.09468842],
       ...,
       [105.59033966, 151.08242798,  64.61215973],
       [106.18034363, 149.52819824,  64.34603119],
       [105.53399658, 150.41226196,  62.96197128]])

In [17]:

from tblite.interface import Calculator

numbers = np.array([atom.atomic_number for atom in mol.atoms])
positions = mol.conformers[0].m

calc = Calculator(
    method="GFN2-xTB",
    numbers=numbers,
    positions=positions,
)
calc.set('save-integrals', 1)
res = calc.singlepoint()
S_mat = res.get('overlap-matrix')
H_mat = res.get('hamiltonian-matrix')

------------------------------------------------------------
  cycle        total energy    energy error   density error
------------------------------------------------------------
      1      57.21831038599  -4.2833039E+02   5.9511351E-01
      2      54.68673176635  -2.5315786E+00   3.0231521E-01
      3      102.5958102302   4.7909078E+01   2.6083096E-01
      4      105.3795070570   2.7836968E+00   2.0462745E-01
      5      85.41514583511  -1.9964361E+01   1.7169666E-01
      6      49.00883299856  -3.6406313E+01   8.3713708E-02
      7      56.32894944671   7.3201164E+00   7.3079263E-02
      8      62.27967254279   5.9507231E+00   8.2362534E-02
      9      47.85408211164  -1.4425590E+01   4.7535651E-02
     10      45.64213424545  -2.2119479E+00   3.8889794E-02
     11      44.06991551298  -1.5722187E+00   2.2194440E-02
     12      43.77102245850  -2.9889305E-01   1.7076444E-02
     13      44.20989510477   4.3887265E-01   1.8717085E-02
     14      43.74778197772  -4.621131


 iter      E             dE          RMSdq      gap      omega  full diag
   1   -431.2651078 -0.431265E+03  0.192E+01    0.19       0.0  T
   2   -431.5061831 -0.241075E+00  0.961E+00    0.51       1.0  T
   3   -371.9661008  0.595401E+02  0.894E+00    0.06       1.0  T
   4   -391.4756737 -0.195096E+02  0.645E+00    0.07       1.0  T
   5   -432.8189641 -0.413433E+02  0.373E+00    0.11       1.0  T
   6   -421.2732293  0.115457E+02  0.339E+00    0.04       1.0  T
   7   -422.6568401 -0.138361E+01  0.313E+00    0.11       1.0  T
   8   -432.2719046 -0.961506E+01  0.237E+00    0.05       1.0  T
   9   -438.6579522 -0.638605E+01  0.135E+00    0.22       1.0  T
  10   -440.7890410 -0.213109E+01  0.105E+00    0.14       1.0  T
  11   -441.3664959 -0.577455E+00  0.803E-01    0.12       1.0  T
  12   -440.8291328  0.537363E+00  0.826E-01    0.05       1.0  T
  13   -441.2306727 -0.401540E+00  0.710E-01    0.13       1.0  T
  14   -441.6621432 -0.431471E+00  0.477E-01    0.15       1.0  T
 

In [ ]:
res.get_orbital_eigenvalues()

array([-1.18359599, -1.17512779, -1.16068435, ..., 52.10569017,
       52.66710544, 53.64378493])

## Running the coupling matrix calculation

The coupling matrix between fragments is calculated using the `kgp.coupling_matrix` function.

Here we are calculating the coupling matrix for fragments in the Universe `u` where
- coupling is calculated between fragments with a closest approach of less than 5.0 Angstrom (`nn_cutoff`)
- coupling is calculated between the LUMO upwards (`state='lumo'`)
- one state per fragment is considered (`degeneracy=1`)
- we will analyse up to frame 3 (`stop=3`)

This function will (for each frame)
- identify which fragments are close enough to possibly be electronically coupled
- run a tight binding calculation between all pairs identified
- calculate the molecular coupling based on this tight binding calculation

In [ ]:
res = kgp.coupling_matrix(u, nn_cutoff=5.0, state='lumo', degeneracy=1, stop=3, model='ocelotml', client=True)

/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60825 instead
  warnings.warn(
2025-06-05T16:08:09.655317+0100 INFO Processing 3 frames
2025-06-05T16:08:09.657143+0100 INFO Processing frame 1 of 3
/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"
2025-06-05T16:08:10.030477+0100 INFO Finding dimers within 5.0, passed 250 fragments
2025-06-05T16:08:10.387019+0100 INF

The `res` object is a namedtuple which contains all the data necessary to perform further analysis.
This object has various attributes which will not be briefly explained.

The `.frames` attribute records which frames from the trajectory were analysed.
This is useful to later cross reference data with the original MD trajectory data.

In [ ]:
print(res.frames)

[0 1 2]


The `.degeneracy` attribute stores how many degenerate states were considered for each fragment.
This value will not change over time, so this array has shape `nfragments`.

In this example only a single state per fragment was considered. 

In [ ]:
print(res.degeneracy)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


The `.H_frag` attribute contains the molecular coupling values, stored inside a 3d numpy array.
The first dimension is along the number of frames (quasi time axis),
while the other two move along fragments in the system.

For example `res.H_frag[0, 1, 71]` gives the coupling (in eV) between the 2nd and 13th fragments in the first frame.

In [ ]:
print(res.H_frag.shape)

print(res.H_frag[2, 1, 71])

(3, 250, 250)
0.0002765886942250739


Producing these results is often a time consuming part of the analysis,
therefore it is wise to save them to a file so you can come back to them later!

This can be done using the `kugupu.save_results` function, which will save the results to a hdf5 (compressed) format.

In [ ]:
kgp.save_results('myresults.hdf5', res)

These results can then be retrieved again using the `kugupu.load_results` function:

In [ ]:
kgp.load_results('./myresults.hdf5')

KugupuResults(frames=array([0, 1, 2]), H_frag=array([[[-10.27936597,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        , -10.32038834,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        , -10.35344287, ...,   0.        ,
           0.        ,   0.        ],
        ...,
        [  0.        ,   0.        ,   0.        , ..., -10.43146138,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
         -10.50477574,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        , -10.37584228]],

       [[-10.38898008,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        , -10.43337746,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        , -10.44523772, ...,   0.        ,
     